In [1]:
# import findspark
import findspark
# initialize findspark with spark directory
findspark.init("C:\BigData\BigData\spark-3.1.2-bin-hadoop3.2")
# import pyspark
import pyspark
# create spark context
sc = pyspark.SparkContext()
# create spark session 
spark = pyspark.sql.SparkSession(sc)

In [2]:
# check
spark

In [13]:
# import packages
import os 
import pickle

import re
from datetime import datetime

import requests

import pytz
import emoji

import pandas as pd
import numpy as np

import ast

import pyspark.sql.functions as F
from pyspark.sql.types import *

# 1. Import Data & Exploration

In [3]:
#set this path to your path, for some reason I have an error 
#reading in all the files
path_json = ".././../data/Topic_vegan/*.json"

In [4]:
df_json = spark.read.json(path_json)

In [5]:
# check the schema of our json dataframe
df_json.printSchema()

root
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- id_str: string (nullable = true)
 |    |    |    |

In [15]:
# select interesting features
df = df_json.select(F.col("user.name"),
                                F.col("user.screen_name"),
                                F.col("created_at"), 
                                F.col("full_text"),
                                F.col("entities.hashtags"),
                                F.col("lang"),
                                F.col("favorite_count"),
                                F.col("user.followers_count"),
                                F.col("user.friends_count"),
                                F.col("user.favourites_count"),
                                F.col("entities.urls"),
                                F.col("entities.symbols"))

# 2. Preprocess Data

## 2.1 Check Time Period 

In [16]:
# https://developer.twitter.com/en/docs/twitter-ads-api/timezones
# function to convert Twitter date string format
def getDate(date):
    if date is not None:
        return str(datetime.strptime(date,'%a %b %d %H:%M:%S +0000 %Y').replace(tzinfo=pytz.UTC).strftime("%Y-%m-%d %H:%M:%S"))
    else:
        return None

# UDF declaration
date_udf = F.udf(getDate, StringType())

# apply udf
df = df.withColumn('post_created_at', F.to_utc_timestamp(date_udf("created_at"), "UTC"))

In [18]:
# show
df.select("created_at", "post_created_at").show(5, truncate=False)

+------------------------------+-------------------+
|created_at                    |post_created_at    |
+------------------------------+-------------------+
|Mon Apr 04 10:09:55 +0000 2022|2022-04-04 10:09:55|
|Mon Apr 04 10:09:54 +0000 2022|2022-04-04 10:09:54|
|Mon Apr 04 10:09:54 +0000 2022|2022-04-04 10:09:54|
|Mon Apr 04 10:09:52 +0000 2022|2022-04-04 10:09:52|
|Mon Apr 04 10:09:52 +0000 2022|2022-04-04 10:09:52|
+------------------------------+-------------------+
only showing top 5 rows

